**Source of the materials**: Biopython cookbook (adapted)
<font color='red'>Status: Draft</font>

BLAST and other sequence search tools (*experimental code*) {#chapter:searchio}
===========================================================

*WARNING*: This chapter of the Tutorial describes an *experimental*
module in Biopython. It is being included in Biopython and documented
here in the tutorial in a pre-final state to allow a period of feedback
and refinement before we declare it stable. Until then the details will
probably change, and any scripts using the current `Bio.SearchIO` would
need to be updated. Please keep this in mind! For stable code working
with NCBI BLAST, please continue to use Bio.Blast described in the
preceding Chapter \[chapter:blast\].

Biological sequence identification is an integral part of
bioinformatics. Several tools are available for this, each with their
own algorithms and approaches, such as BLAST (arguably the most
popular), FASTA, HMMER, and many more. In general, these tools usually
use your sequence to search a database of potential matches. With the
growing number of known sequences (hence the growing number of potential
matches), interpreting the results becomes increasingly hard as there
could be hundreds or even thousands of potential matches. Naturally,
manual interpretation of these searches’ results is out of the question.
Moreover, you often need to work with several sequence search tools,
each with its own statistics, conventions, and output format. Imagine
how daunting it would be when you need to work with multiple sequences
using multiple search tools.

We know this too well ourselves, which is why we created the
`Bio.SearchIO` submodule in Biopython. `Bio.SearchIO` allows you to
extract information from your search results in a convenient way, while
also dealing with the different standards and conventions used by
different search tools. The name `SearchIO` is a homage to BioPerl’s
module of the same name.

In this chapter, we’ll go through the main features of `Bio.SearchIO` to
show what it can do for you. We’ll use two popular search tools along
the way: BLAST and BLAT. They are used merely for illustrative purposes,
and you should be able to adapt the workflow to any other search tools
supported by `Bio.SearchIO` in a breeze. You’re very welcome to follow
along with the search output files we’ll be using. The BLAST output file
can be downloaded
[here](http://biopython.org/SRC/Tests/Tutorial/my_blast.xml), and the
BLAT output file
[here](http://biopython.org/SRC/Tests/Tutorial/my_blat.psl). Both output
files were generated using this sequence:



```
>mystery_seq
CCCTCTACAGGGAAGCGCTTTCTGTTGTCTGAAAGAAAAGAAAGTGCTTCCTTTTAGAGGG

```


The BLAST result is an XML file generated using `blastn` against the
NCBI `refseq_rna` database. For BLAT, the sequence database was the
February 2009 `hg19` human genome draft and the output format is PSL.

We’ll start from an introduction to the `Bio.SearchIO` object model. The
model is the representation of your search results, thus it is core to
`Bio.SearchIO` itself. After that, we’ll check out the main functions in
`Bio.SearchIO` that you may often use.

Now that we’re all set, let’s go to the first step: introducing the core
object model.

The SearchIO object model {#sec:searchio-model}
-------------------------

Despite the wildly differing output styles among many sequence search
tools, it turns out that their underlying concept is similar:

-   The output file may contain results from one or more search queries.

-   In each search query, you will see one or more hits from the given
    search database.

-   In each database hit, you will see one or more regions containing
    the actual sequence alignment between your query sequence and the
    database sequence.

-   Some programs like BLAT or Exonerate may further split these regions
    into several alignment fragments (or blocks in BLAT and possibly
    exons in exonerate). This is not something you always see, as
    programs like BLAST and HMMER do not do this.

Realizing this generality, we decided use it as base for creating the
`Bio.SearchIO` object model. The object model consists of a nested
hierarchy of Python objects, each one representing one concept outlined
above. These objects are:

-   `QueryResult`, to represent a single search query.

-   `Hit`, to represent a single database hit. `Hit` objects are
    contained within `QueryResult` and in each `QueryResult` there is
    zero or more `Hit` objects.

-   `HSP` (short for high-scoring pair), to represent region(s) of
    significant alignments between query and hit sequences. `HSP`
    objects are contained within `Hit` objects and each `Hit` has one or
    more `HSP` objects.

-   `HSPFragment`, to represent a single contiguous alignment between
    query and hit sequences. `HSPFragment` objects are contained within
    `HSP` objects. Most sequence search tools like BLAST and HMMER unify
    `HSP` and `HSPFragment` objects as each `HSP` will only have a
    single `HSPFragment`. However there are tools like BLAT and
    Exonerate that produce `HSP` containing multiple `HSPFragment`.
    Don’t worry if this seems a tad confusing now, we’ll elaborate more
    on these two objects later on.

These four objects are the ones you will interact with when you use
`Bio.SearchIO`. They are created using one of the main `Bio.SearchIO`
methods: `read`, `parse`, `index`, or `index_db`. The details of these
methods are provided in later sections. For this section, we’ll only be
using read and parse. These functions behave similarly to their
`Bio.SeqIO` and `Bio.AlignIO` counterparts:

-   `read` is used for search output files with a single query and
    returns a `QueryResult` object

-   `parse` is used for search output files with multiple queries and
    returns a generator that yields `QueryResult` objects

With that settled, let’s start probing each `Bio.SearchIO` object,
beginning with `QueryResult`.

### QueryResult {#sec:searchio-qresult}

The QueryResult object represents a single search query and contains
zero or more Hit objects. Let’s see what it looks like using the BLAST
file we have:



In [1]:
from Bio import SearchIO
blast_qresult = SearchIO.read('data/my_blast.xml', 'blast-xml')
print(blast_qresult)

Program: blastn (2.3.0+)
  Query: gi|8332116|gb|BE037100.1|BE037100 (1111)
         MP14H09 MP Mesembryanthemum crystallinum cDNA 5' similar to cold ac...
 Target: nt
   Hits: ----  -----  ----------------------------------------------------------
            #  # HSP  ID + description
         ----  -----  ----------------------------------------------------------
            0      1  gi|731339628|ref|XM_010682658.1|  PREDICTED: Beta vulga...
            1      1  gi|568824607|ref|XM_006466626.1|  PREDICTED: Citrus sin...
            2      1  gi|568824605|ref|XM_006466625.1|  PREDICTED: Citrus sin...
            3      1  gi|568824603|ref|XM_006466624.1|  PREDICTED: Citrus sin...
            4      1  gi|568824601|ref|XM_006466623.1|  PREDICTED: Citrus sin...
            5      1  gi|568824599|ref|XM_006466622.1|  PREDICTED: Citrus sin...
            6      1  gi|567866318|ref|XM_006425719.1|  Citrus clementina hyp...
            7      1  gi|567866316|ref|XM_006425718.1|  Citrus cl

/home/tiago_antao/miniconda/lib/python3.5/site-packages/Bio/SearchIO/__init__.py:211: BiopythonExperimentalWarning: Bio.SearchIO is an experimental submodule which may undergo significant changes prior to its future official release.
  BiopythonExperimentalWarning)



We’ve just begun to scratch the surface of the object model, but you can
see that there’s already some useful information. By invoking `print` on
the `QueryResult` object, you can see:

-   The program name and version (blastn version 2.2.27+)

-   The query ID, description, and its sequence length (ID is 42291,
    description is ‘mystery\_seq’, and it is 61 nucleotides long)

-   The target database to search against (refseq\_rna)

-   A quick overview of the resulting hits. For our query sequence,
    there are 100 potential hits (numbered 0–99 in the table). For each
    hit, we can also see how many HSPs it contains, its ID, and a
    snippet of its description. Notice here that `Bio.SearchIO`
    truncates the hit table overview, by showing only hits numbered
    0–29, and then 97–99.

Now let’s check our BLAT results using the same procedure as above:



In [2]:
blat_qresult = SearchIO.read('data/my_blat.psl', 'blat-psl')
print(blat_qresult)

Program: blat (<unknown version>)
  Query: mystery_seq (61)
         <unknown description>
 Target: <unknown target>
   Hits: ----  -----  ----------------------------------------------------------
            #  # HSP  ID + description
         ----  -----  ----------------------------------------------------------
            0     17  chr19  <unknown description>



You’ll immediately notice that there are some differences. Some of these
are caused by the way PSL format stores its details, as you’ll see. The
rest are caused by the genuine program and target database differences
between our BLAST and BLAT searches:

-   The program name and version. `Bio.SearchIO` knows that the program
    is BLAT, but in the output file there is no information regarding
    the program version so it defaults to ‘&lt;unknown version&gt;’.

-   The query ID, description, and its sequence length. Notice here that
    these details are slightly different from the ones we saw in BLAST.
    The ID is ‘mystery\_seq’ instead of 42991, there is no known
    description, but the query length is still 61. This is actually a
    difference introduced by the file formats themselves. BLAST
    sometimes creates its own query IDs and uses your original ID as the
    sequence description.

-   The target database is not known, as it is not stated in the BLAT
    output file.

-   And finally, the list of hits we have is completely different. Here,
    we see that our query sequence only hits the ‘chr19’ database entry,
    but in it we see 17 HSP regions. This should not be surprising
    however, given that we are using a different program, each with its
    own target database.

All the details you saw when invoking the `print` method can be accessed
individually using Python’s object attribute access notation (a.k.a. the
dot notation). There are also other format-specific attributes that you
can access using the same method.



In [3]:
print("%s %s" % (blast_qresult.program, blast_qresult.version))

blastn 2.3.0+


In [4]:
print("%s %s" % (blat_qresult.program, blat_qresult.version))

blat <unknown version>


In [5]:
blast_qresult.param_evalue_threshold    # blast-xml specific

10.0


For a complete list of accessible attributes, you can check each
format-specific documentation. Here are the ones [for
BLAST](http://biopython.org/DIST/docs/api/Bio.SearchIO.BlastIO-module.html)
and for
[BLAT](http://biopython.org/DIST/docs/api/Bio.SearchIO.BlatIO-module.html).

Having looked at using `print` on `QueryResult` objects, let’s drill
down deeper. What exactly is a `QueryResult`? In terms of Python
objects, `QueryResult` is a hybrid between a list and a dictionary. In
other words, it is a container object with all the convenient features
of lists and dictionaries.

Like Python lists and dictionaries, `QueryResult` objects are iterable.
Each iteration returns a `Hit` object:



In [6]:
for hit in blast_qresult:
    hit


To check how many items (hits) a `QueryResult` has, you can simply
invoke Python’s `len` method:



In [7]:
len(blast_qresult)

50

In [8]:
len(blat_qresult)

1


Like Python lists, you can retrieve items (hits) from a `QueryResult`
using the slice notation:



In [9]:
blast_qresult[0]        # retrieves the top hit

Hit(id='gi|731339628|ref|XM_010682658.1|', query_id='gi|8332116|gb|BE037100.1|BE037100', 1 hsps)

In [10]:
blast_qresult[-1]       # retrieves the last hit

Hit(id='gi|255762732|gb|GQ370517.1|', query_id='gi|8332116|gb|BE037100.1|BE037100', 1 hsps)


To retrieve multiple hits, you can slice `QueryResult` objects using the
slice notation as well. In this case, the slice will return a new
`QueryResult` object containing only the sliced hits:



In [11]:
blast_slice = blast_qresult[:3]     # slices the first three hits
print(blast_slice)

Program: blastn (2.3.0+)
  Query: gi|8332116|gb|BE037100.1|BE037100 (1111)
         MP14H09 MP Mesembryanthemum crystallinum cDNA 5' similar to cold ac...
 Target: nt
   Hits: ----  -----  ----------------------------------------------------------
            #  # HSP  ID + description
         ----  -----  ----------------------------------------------------------
            0      1  gi|731339628|ref|XM_010682658.1|  PREDICTED: Beta vulga...
            1      1  gi|568824607|ref|XM_006466626.1|  PREDICTED: Citrus sin...
            2      1  gi|568824605|ref|XM_006466625.1|  PREDICTED: Citrus sin...



Like Python dictionaries, you can also retrieve hits using the hit’s ID.
This is particularly useful if you know a given hit ID exists within a
search query results:



In [16]:
blast_qresult['gi|731339628|ref|XM_010682658.1|']

Hit(id='gi|731339628|ref|XM_010682658.1|', query_id='gi|8332116|gb|BE037100.1|BE037100', 1 hsps)


You can also get a full list of `Hit` objects using `hits` and a full
list of `Hit` IDs using `hit_keys`:



In [17]:
blast_qresult.hits

[Hit(id='gi|731339628|ref|XM_010682658.1|', query_id='gi|8332116|gb|BE037100.1|BE037100', 1 hsps),
 Hit(id='gi|568824607|ref|XM_006466626.1|', query_id='gi|8332116|gb|BE037100.1|BE037100', 1 hsps),
 Hit(id='gi|568824605|ref|XM_006466625.1|', query_id='gi|8332116|gb|BE037100.1|BE037100', 1 hsps),
 Hit(id='gi|568824603|ref|XM_006466624.1|', query_id='gi|8332116|gb|BE037100.1|BE037100', 1 hsps),
 Hit(id='gi|568824601|ref|XM_006466623.1|', query_id='gi|8332116|gb|BE037100.1|BE037100', 1 hsps),
 Hit(id='gi|568824599|ref|XM_006466622.1|', query_id='gi|8332116|gb|BE037100.1|BE037100', 1 hsps),
 Hit(id='gi|567866318|ref|XM_006425719.1|', query_id='gi|8332116|gb|BE037100.1|BE037100', 1 hsps),
 Hit(id='gi|567866316|ref|XM_006425718.1|', query_id='gi|8332116|gb|BE037100.1|BE037100', 1 hsps),
 Hit(id='gi|567866314|ref|XM_006425717.1|', query_id='gi|8332116|gb|BE037100.1|BE037100', 1 hsps),
 Hit(id='gi|567866312|ref|XM_006425716.1|', query_id='gi|8332116|gb|BE037100.1|BE037100', 1 hsps),
 Hit(id='g

In [18]:
blast_qresult.hit_keys

['gi|731339628|ref|XM_010682658.1|',
 'gi|568824607|ref|XM_006466626.1|',
 'gi|568824605|ref|XM_006466625.1|',
 'gi|568824603|ref|XM_006466624.1|',
 'gi|568824601|ref|XM_006466623.1|',
 'gi|568824599|ref|XM_006466622.1|',
 'gi|567866318|ref|XM_006425719.1|',
 'gi|567866316|ref|XM_006425718.1|',
 'gi|567866314|ref|XM_006425717.1|',
 'gi|567866312|ref|XM_006425716.1|',
 'gi|590704208|ref|XM_007047033.1|',
 'gi|590704205|ref|XM_007047032.1|',
 'gi|694428700|ref|XM_009343631.1|',
 'gi|694402986|ref|XM_009378191.1|',
 'gi|743838297|ref|XM_011027373.1|',
 'gi|743838293|ref|XM_011027372.1|',
 'gi|595807351|ref|XM_007202530.1|',
 'gi|566180892|ref|XM_006380679.1|',
 'gi|764593175|ref|XM_004300526.2|',
 'gi|731440276|ref|XM_002274845.3|',
 'gi|349709091|emb|FQ378501.1|',
 'gi|719997221|ref|XM_010256725.1|',
 'gi|645272858|ref|XM_008243375.1|',
 'gi|645272856|ref|XM_008243374.1|',
 'gi|848856318|ref|XM_013000712.1|',
 'gi|255562758|ref|XM_002522339.1|',
 'gi|658006068|ref|XM_008339966.1|',
 'gi|


What if you just want to check whether a particular hit is present in
the query results? You can do a simple Python membership test using the
`in` keyword:



In [19]:
'gi|262205317|ref|NR_030195.1|' in blast_qresult

False

In [21]:
'gi|731339628|ref|XM_010682658.1|' in blast_qresult

True


Sometimes, knowing whether a hit is present is not enough; you also want
to know the rank of the hit. Here, the `index` method comes to the
rescue:



In [23]:
blast_qresult.index('gi|595807351|ref|XM_007202530.1|')

16


Remember that we’re using Python’s indexing style here, which is
zero-based. This means our hit above is ranked at no. 23, not 22.

Also, note that the hit rank you see here is based on the native hit
ordering present in the original search output file. Different search
tools may order these hits based on different criteria.

If the native hit ordering doesn’t suit your taste, you can use the
`sort` method of the `QueryResult` object. It is very similar to
Python’s `list.sort` method, with the addition of an option to create a
new sorted `QueryResult` object or not.

Here is an example of using `QueryResult.sort` to sort the hits based on
each hit’s full sequence length. For this particular sort, we’ll set the
`in_place` flag to `False` so that sorting will return a new
`QueryResult` object and leave our initial object unsorted. We’ll also
set the `reverse` flag to `True` so that we sort in descending order.



In [24]:
for hit in blast_qresult[:5]:   # id and sequence length of the first five hits
    print("%s %i" % (hit.id, hit.seq_len))

gi|731339628|ref|XM_010682658.1| 847
gi|568824607|ref|XM_006466626.1| 878
gi|568824605|ref|XM_006466625.1| 911
gi|568824603|ref|XM_006466624.1| 894
gi|568824601|ref|XM_006466623.1| 922


In [25]:
sort_key = lambda hit: hit.seq_len
sorted_qresult = blast_qresult.sort(key=sort_key, reverse=True, in_place=False)
for hit in sorted_qresult[:5]:
    print("%s %i" % (hit.id, hit.seq_len))

gi|955306249|ref|XM_003519866.3| 1388
gi|659079295|ref|XM_008441960.1| 1245
gi|567866316|ref|XM_006425718.1| 1202
gi|566180892|ref|XM_006380679.1| 1182
gi|565355903|ref|XM_006344753.1| 1153



The advantage of having the `in_place` flag here is that we’re
preserving the native ordering, so we may use it again later. You should
note that this is not the default behavior of `QueryResult.sort`,
however, which is why we needed to set the `in_place` flag to `True`
explicitly.

At this point, you’ve known enough about `QueryResult` objects to make
it work for you. But before we go on to the next object in the
`Bio.SearchIO` model, let’s take a look at two more sets of methods that
could make it even easier to work with `QueryResult` objects: the
`filter` and `map` methods.

If you’re familiar with Python’s list comprehensions, generator
expressions or the built in `filter` and `map` functions, you’ll know
how useful they are for working with list-like objects (if you’re not,
check them out!). You can use these built in methods to manipulate
`QueryResult` objects, but you’ll end up with regular Python lists and
lose the ability to do more interesting manipulations.

That’s why, `QueryResult` objects provide its own flavor of `filter` and
`map` methods. Analogous to `filter`, there are `hit_filter` and
`hsp_filter` methods. As their name implies, these methods filter its
`QueryResult` object either on its `Hit` objects or `HSP` objects.
Similarly, analogous to `map`, `QueryResult` objects also provide the
`hit_map` and `hsp_map` methods. These methods apply a given function to
all hits or HSPs in a `QueryResult` object, respectively.

Let’s see these methods in action, beginning with `hit_filter`. This
method accepts a callback function that checks whether a given `Hit`
object passes the condition you set or not. In other words, the function
must accept as its argument a single `Hit` object and returns `True` or
`False`.

Here is an example of using `hit_filter` to filter out `Hit` objects
that only have one HSP:



In [26]:
filter_func = lambda hit: len(hit.hsps) > 1     # the callback function
len(blast_qresult)      # no. of hits before filtering

50

In [27]:
filtered_qresult = blast_qresult.hit_filter(filter_func)
len(filtered_qresult)   # no. of hits after filtering

0

In [28]:
for hit in filtered_qresult[:5]:    # quick check for the hit lengths
    print("%s %i" % (hit.id, len(hit.hsps)))


`hsp_filter` works the same as `hit_filter`, only instead of looking at
the `Hit` objects, it performs filtering on the `HSP` objects in each
hits.

As for the `map` methods, they too accept a callback function as their
arguments. However, instead of returning `True` or `False`, the callback
function must return the modified `Hit` or `HSP` object (depending on
whether you’re using `hit_map` or `hsp_map`).

Let’s see an example where we’re using `hit_map` to rename the hit IDs:



In [29]:
def map_func(hit):
    hit.id = hit.id.split('|')[3]   # renames 'gi|301171322|ref|NR_035857.1|' to 'NR_035857.1'
    return hit

mapped_qresult = blast_qresult.hit_map(map_func)
for hit in mapped_qresult[:5]:
    print(hit.id)

XM_010682658.1
XM_006466626.1
XM_006466625.1
XM_006466624.1
XM_006466623.1



Again, `hsp_map` works the same as `hit_map`, but on `HSP` objects
instead of `Hit` objects.

### Hit {#sec:searchio-hit}

`Hit` objects represent all query results from a single database entry.
They are the second-level container in the `Bio.SearchIO` object
hierarchy. You’ve seen that they are contained by `QueryResult` objects,
but they themselves contain `HSP` objects.

Let’s see what they look like, beginning with our BLAST search:



In [30]:
from Bio import SearchIO
blast_qresult = SearchIO.read('data/my_blast.xml', 'blast-xml')
blast_hit = blast_qresult[3]    # fourth hit from the query result
print(blast_hit)

Query: gi|8332116|gb|BE037100.1|BE037100
       MP14H09 MP Mesembryanthemum crystallinum cDNA 5' similar to cold accl...
  Hit: gi|568824603|ref|XM_006466624.1| (894)
       PREDICTED: Citrus sinensis cold-regulated 413 plasma membrane protein...
 HSPs: ----  --------  ---------  ------  ---------------  ---------------------
          #   E-value  Bit score    Span      Query range              Hit range
       ----  --------  ---------  ------  ---------------  ---------------------
          0   6.9e-99     372.78     596         [63:655]              [108:697]



You see that we’ve got the essentials covered here:

-   The query ID and description is present. A hit is always tied to a
    query, so we want to keep track of the originating query as well.
    These values can be accessed from a hit using the `query_id` and
    `query_description` attributes.

-   We also have the unique hit ID, description, and full
    sequence lengths. They can be accessed using `id`, `description`,
    and `seq_len`, respectively.

-   Finally, there’s a table containing quick information about the HSPs
    this hit contains. In each row, we’ve got the important HSP details
    listed: the HSP index, its e-value, its bit score, its span (the
    alignment length including gaps), its query coordinates, and its
    hit coordinates.

Now let’s contrast this with the BLAT search. Remember that in the BLAT
search we had one hit with 17 HSPs.



In [31]:
blat_qresult = SearchIO.read('data/my_blat.psl', 'blat-psl')
blat_hit = blat_qresult[0]      # the only hit
print(blat_hit)

Query: mystery_seq
       <unknown description>
  Hit: chr19 (59128983)
       <unknown description>
 HSPs: ----  --------  ---------  ------  ---------------  ---------------------
          #   E-value  Bit score    Span      Query range              Hit range
       ----  --------  ---------  ------  ---------------  ---------------------
          0         ?          ?       ?           [0:61]    [54204480:54204541]
          1         ?          ?       ?           [0:61]    [54233104:54264463]
          2         ?          ?       ?           [0:61]    [54254477:54260071]
          3         ?          ?       ?           [1:61]    [54210720:54210780]
          4         ?          ?       ?           [0:60]    [54198476:54198536]
          5         ?          ?       ?           [0:61]    [54265610:54265671]
          6         ?          ?       ?           [0:61]    [54238143:54240175]
          7         ?          ?       ?           [0:60]    [54189735:54189795]
        


Here, we’ve got a similar level of detail as with the BLAST hit we saw
earlier. There are some differences worth explaining, though:

-   The e-value and bit score column values. As BLAT HSPs do not have
    e-values and bit scores, the display defaults to ‘?’.

-   What about the span column? The span values is meant to display the
    complete alignment length, which consists of all residues and any
    gaps that may be present. The PSL format do not have this
    information readily available and `Bio.SearchIO` does not attempt to
    try guess what it is, so we get a ‘?’ similar to the e-value and bit
    score columns.

In terms of Python objects, `Hit` behaves almost the same as Python
lists, but contain `HSP` objects exclusively. If you’re familiar with
lists, you should encounter no difficulties working with the `Hit`
object.

Just like Python lists, `Hit` objects are iterable, and each iteration
returns one `HSP` object it contains:



In [32]:
for hsp in blast_hit:
    hsp


You can invoke `len` on a `Hit` to see how many `HSP` objects it has:



In [33]:
len(blast_hit)

1

In [34]:
len(blat_hit)

17


You can use the slice notation on `Hit` objects, whether to retrieve
single `HSP` or multiple `HSP` objects. Like `QueryResult`, if you slice
for multiple `HSP`, a new `Hit` object will be returned containing only
the sliced `HSP` objects:



In [35]:
blat_hit[0]                 # retrieve single items

HSP(hit_id='chr19', query_id='mystery_seq', 1 fragments)

In [36]:
sliced_hit = blat_hit[4:9]  # retrieve multiple items
len(sliced_hit)

5

In [37]:
print(sliced_hit)

Query: mystery_seq
       <unknown description>
  Hit: chr19 (59128983)
       <unknown description>
 HSPs: ----  --------  ---------  ------  ---------------  ---------------------
          #   E-value  Bit score    Span      Query range              Hit range
       ----  --------  ---------  ------  ---------------  ---------------------
          0         ?          ?       ?           [0:60]    [54198476:54198536]
          1         ?          ?       ?           [0:61]    [54265610:54265671]
          2         ?          ?       ?           [0:61]    [54238143:54240175]
          3         ?          ?       ?           [0:60]    [54189735:54189795]
          4         ?          ?       ?           [0:61]    [54185425:54185486]



You can also sort the `HSP` inside a `Hit`, using the exact same
arguments like the sort method you saw in the `QueryResult` object.

Finally, there are also the `filter` and `map` methods you can use on
`Hit` objects. Unlike in the `QueryResult` object, `Hit` objects only
have one variant of `filter` (`Hit.filter`) and one variant of `map`
(`Hit.map`). Both of `Hit.filter` and `Hit.map` work on the `HSP`
objects a `Hit` has.

### HSP {#sec:searchio-hsp}

`HSP` (high-scoring pair) represents region(s) in the hit sequence that
contains significant alignment(s) to the query sequence. It contains the
actual match between your query sequence and a database entry. As this
match is determined by the sequence search tool’s algorithms, the `HSP`
object contains the bulk of the statistics computed by the search tool.
This also makes the distinction between `HSP` objects from different
search tools more apparent compared to the differences you’ve seen in
`QueryResult` or `Hit` objects.

Let’s see some examples from our BLAST and BLAT searches. We’ll look at
the BLAST HSP first:



In [38]:
from Bio import SearchIO
blast_qresult = SearchIO.read('data/my_blast.xml', 'blast-xml')
blast_hsp = blast_qresult[0][0]    # first hit, first hsp
print(blast_hsp)

      Query: gi|8332116|gb|BE037100.1|BE037100 MP14H09 MP Mesembryanthemum cr...
        Hit: gi|731339628|ref|XM_010682658.1| PREDICTED: Beta vulgaris subsp....
Query range: [86:679] (1)
  Hit range: [80:677] (1)
Quick stats: evalue 1.2e-108; bitscore 405.24
  Fragments: 1 (597 columns)
     Query - TTGGCCATGAAAACTGATCAATTGGCCGTGGCTAATATGATCGATTCCGATATCAATGA~~~TGTAG
             ||||||||||||||||| ||| ||||  |||||||| |||| ||||  ||||| |||||~~~|||||
       Hit - TTGGCCATGAAAACTGAGCAAATGGCGTTGGCTAATTTGATAGATTATGATATGAATGA~~~TGTAG



Just like `QueryResult` and `Hit`, invoking `print` on an `HSP` shows
its general details:

-   There are the query and hit IDs and descriptions. We need these to
    identify our `HSP`.

-   We’ve also got the matching range of the query and hit sequences.
    The slice notation we’re using here is an indication that the range
    is displayed using Python’s indexing style (zero-based, half open).
    The number inside the parenthesis denotes the strand. In this case,
    both sequences have the plus strand.

-   Some quick statistics are available: the e-value and bitscore.

-   There is information about the HSP fragments. Ignore this for now;
    it will be explained later on.

-   And finally, we have the query and hit sequence alignment itself.

These details can be accessed on their own using the dot notation, just
like in `QueryResult` and `Hit`:



In [39]:
blast_hsp.query_range

(86, 679)

In [40]:
blast_hsp.evalue

1.1684e-108


They’re not the only attributes available, though. `HSP` objects come
with a default set of properties that makes it easy to probe their
various details. Here are some examples:



In [41]:
blast_hsp.hit_start         # start coordinate of the hit sequence

80

In [42]:
def map_func(hit):
    hit.id = hit.id.split('|')[3]   # renames 'gi|301171322|ref|NR_035857.1|' to 'NR_035857.1'
    return hitblast_hsp.query_span        # how many residues in the query sequence

In [43]:
blast_hsp.aln_span          # how long the alignment is

597


Check out the `HSP`
[documentation](http://biopython.org/DIST/docs/api/Bio.SearchIO._model.hsp-module.html)
for a full list of these predefined properties.

Furthermore, each sequence search tool usually computes its own
statistics / details for its `HSP` objects. For example, an XML BLAST
search also outputs the number of gaps and identical residues. These
attributes can be accessed like so:



In [44]:
blast_hsp.gap_num       # number of gaps

4

In [45]:
blast_hsp.ident_num     # number of identical residues

449


These details are format-specific; they may not be present in other
formats. To see which details are available for a given sequence search
tool, you should check the format’s documentation in `Bio.SearchIO`.
Alternatively, you may also use `.__dict__.keys()` for a quick list of
what’s available:



In [46]:
blast_hsp.__dict__.keys()

dict_keys(['_items', 'pos_num', 'gap_num', 'bitscore', 'bitscore_raw', 'ident_num', 'evalue'])


Finally, you may have noticed that the `query` and `hit` attributes of
our HSP are not just regular strings:



In [47]:
blast_hsp.query

SeqRecord(seq=Seq('TTGGCCATGAAAACTGATCAATTGGCCGTGGCTAATATGATCGATTCCGATATC...TAG', DNAAlphabet()), id='gi|8332116|gb|BE037100.1|BE037100', name='aligned query sequence', description="MP14H09 MP Mesembryanthemum crystallinum cDNA 5' similar to cold acclimation protein, mRNA sequence", dbxrefs=[])

In [48]:
blast_hsp.hit

SeqRecord(seq=Seq('TTGGCCATGAAAACTGAGCAAATGGCGTTGGCTAATTTGATAGATTATGATATG...TAG', DNAAlphabet()), id='gi|731339628|ref|XM_010682658.1|', name='aligned hit sequence', description='PREDICTED: Beta vulgaris subsp. vulgaris cold-regulated 413 plasma membrane protein 2 (LOC104895996), mRNA', dbxrefs=[])


They are `SeqRecord` objects you saw earlier in
Section \[chapter:SeqRecord\]! This means that you can do all sorts of
interesting things you can do with `SeqRecord` objects on `HSP.query`
and/or `HSP.hit`.

It should not surprise you now that the `HSP` object has an `alignment`
property which is a `MultipleSeqAlignment` object:



In [49]:
print(blast_hsp.aln)

DNAAlphabet() alignment with 2 rows and 597 columns
TTGGCCATGAAAACTGATCAATTGGCCGTGGCTAATATGATCGA...TAG gi|8332116|gb|BE037100.1|BE037100
TTGGCCATGAAAACTGAGCAAATGGCGTTGGCTAATTTGATAGA...TAG gi|731339628|ref|XM_010682658.1|



Having probed the BLAST HSP, let’s now take a look at HSPs from our BLAT
results for a different kind of HSP. As usual, we’ll begin by invoking
`print` on it:



In [50]:
blat_qresult = SearchIO.read('data/my_blat.psl', 'blat-psl')
blat_hsp = blat_qresult[0][0]       # first hit, first hsp
print(blat_hsp)

      Query: mystery_seq <unknown description>
        Hit: chr19 <unknown description>
Query range: [0:61] (1)
  Hit range: [54204480:54204541] (1)
Quick stats: evalue ?; bitscore ?
  Fragments: 1 (? columns)



Some of the outputs you may have already guessed. We have the query and
hit IDs and descriptions and the sequence coordinates. Values for evalue
and bitscore is ‘?’ as BLAT HSPs do not have these attributes. But The
biggest difference here is that you don’t see any sequence alignments
displayed. If you look closer, PSL formats themselves do not have any
hit or query sequences, so `Bio.SearchIO` won’t create any sequence or
alignment objects. What happens if you try to access `HSP.query`,
`HSP.hit`, or `HSP.aln`? You’ll get the default values for these
attributes, which is `None`:



In [51]:
blat_hsp.hit is None

True

In [52]:
blat_hsp.query is None

True

In [53]:
blat_hsp.aln is None

True


This does not affect other attributes, though. For example, you can
still access the length of the query or hit alignment. Despite not
displaying any attributes, the PSL format still have this information so
`Bio.SearchIO` can extract them:



In [54]:
blat_hsp.query_span     # length of query match

61

In [55]:
blat_hsp.hit_span       # length of hit match

61


Other format-specific attributes are still present as well:



In [56]:
blat_hsp.score          # PSL score

61

In [57]:
blat_hsp.mismatch_num   # the mismatch column

0


So far so good? Things get more interesting when you look at another
‘variant’ of HSP present in our BLAT results. You might recall that in
BLAT searches, sometimes we get our results separated into ‘blocks’.
These blocks are essentially alignment fragments that may have some
intervening sequence between them.

Let’s take a look at a BLAT HSP that contains multiple blocks to see how
`Bio.SearchIO` deals with this:



In [58]:
blat_hsp2 = blat_qresult[0][1]      # first hit, second hsp
print(blat_hsp2)

      Query: mystery_seq <unknown description>
        Hit: chr19 <unknown description>
Query range: [0:61] (1)
  Hit range: [54233104:54264463] (1)
Quick stats: evalue ?; bitscore ?
  Fragments: ---  --------------  ----------------------  ----------------------
               #            Span             Query range               Hit range
             ---  --------------  ----------------------  ----------------------
               0               ?                  [0:18]     [54233104:54233122]
               1               ?                 [18:61]     [54264420:54264463]



What’s happening here? We still some essential details covered: the IDs
and descriptions, the coordinates, and the quick statistics are similar
to what you’ve seen before. But the fragments detail is all different.
Instead of showing ‘Fragments: 1’, we now have a table with two data
rows.

This is how `Bio.SearchIO` deals with HSPs having multiple fragments. As
mentioned before, an HSP alignment may be separated by intervening
sequences into fragments. The intervening sequences are not part of the
query-hit match, so they should not be considered part of query nor hit
sequence. However, they do affect how we deal with sequence coordinates,
so we can’t ignore them.

Take a look at the hit coordinate of the HSP above. In the `Hit range:`
field, we see that the coordinate is `[54233104:54264463]`. But looking
at the table rows, we see that not the entire region spanned by this
coordinate matches our query. Specifically, the intervening region spans
from `54233122` to `54264420`.

Why then, is the query coordinates seem to be contiguous, you ask? This
is perfectly fine. In this case it means that the query match is
contiguous (no intervening regions), while the hit match is not.

All these attributes are accessible from the HSP directly, by the way:



In [59]:
blat_hsp2.hit_range         # hit start and end coordinates of the entire HSP

(54233104, 54264463)

In [60]:
blat_hsp2.hit_range_all     # hit start and end coordinates of each fragment

[(54233104, 54233122), (54264420, 54264463)]

In [61]:
blat_hsp2.hit_span          # hit span of the entire HSP

31359

In [62]:
blat_hsp2.hit_span_all      # hit span of each fragment

[18, 43]

In [63]:
blat_hsp2.hit_inter_ranges  # start and end coordinates of intervening regions in the hit sequence

[(54233122, 54264420)]

In [64]:
blat_hsp2.hit_inter_spans   # span of intervening regions in the hit sequence

[31298]


Most of these attributes are not readily available from the PSL file we
have, but `Bio.SearchIO` calculates them for you on the fly when you
parse the PSL file. All it needs are the start and end coordinates of
each fragment.

What about the `query`, `hit`, and `aln` attributes? If the HSP has
multiple fragments, you won’t be able to use these attributes as they
only fetch single `SeqRecord` or `MultipleSeqAlignment` objects.
However, you can use their `*_all` counterparts: `query_all`, `hit_all`,
and `aln_all`. These properties will return a list containing
`SeqRecord` or `MultipleSeqAlignment` objects from each of the HSP
fragment. There are other attributes that behave similarly, i.e. they
only work for HSPs with one fragment. Check out the `HSP`
[documentation](http://biopython.org/DIST/docs/api/Bio.SearchIO._model.hsp-module.html)
for a full list.

Finally, to check whether you have multiple fragments or not, you can
use the `is_fragmented` property like so:



In [65]:
blat_hsp2.is_fragmented     # BLAT HSP with 2 fragments

True

In [66]:
blat_hsp.is_fragmented      # BLAT HSP from earlier, with one fragment

False


Before we move on, you should also know that we can use the slice
notation on `HSP` objects, just like `QueryResult` or `Hit` objects.
When you use this notation, you’ll get an `HSPFragment` object in
return, the last component of the object model.

### HSPFragment {#sec:searchio-hspfragment}

`HSPFragment` represents a single, contiguous match between the query
and hit sequences. You could consider it the core of the object model
and search result, since it is the presence of these fragments that
determine whether your search have results or not.

In most cases, you don’t have to deal with `HSPFragment` objects
directly since not that many sequence search tools fragment their HSPs.
When you do have to deal with them, what you should remember is that
`HSPFragment` objects were written with to be as compact as possible. In
most cases, they only contain attributes directly related to sequences:
strands, reading frames, alphabets, coordinates, the sequences
themselves, and their IDs and descriptions.

These attributes are readily shown when you invoke `print` on an
`HSPFragment`. Here’s an example, taken from our BLAST search:



In [67]:
from Bio import SearchIO
blast_qresult = SearchIO.read('data/my_blast.xml', 'blast-xml')
blast_frag = blast_qresult[0][0][0]    # first hit, first hsp, first fragment
print(blast_frag)

      Query: gi|8332116|gb|BE037100.1|BE037100 MP14H09 MP Mesembryanthemum cr...
        Hit: gi|731339628|ref|XM_010682658.1| PREDICTED: Beta vulgaris subsp....
Query range: [86:679] (1)
  Hit range: [80:677] (1)
  Fragments: 1 (597 columns)
     Query - TTGGCCATGAAAACTGATCAATTGGCCGTGGCTAATATGATCGATTCCGATATCAATGA~~~TGTAG
             ||||||||||||||||| ||| ||||  |||||||| |||| ||||  ||||| |||||~~~|||||
       Hit - TTGGCCATGAAAACTGAGCAAATGGCGTTGGCTAATTTGATAGATTATGATATGAATGA~~~TGTAG



At this level, the BLAT fragment looks quite similar to the BLAST
fragment, save for the query and hit sequences which are not present:



In [68]:
blat_qresult = SearchIO.read('data/my_blat.psl', 'blat-psl')
blat_frag = blat_qresult[0][0][0]    # first hit, first hsp, first fragment
print(blat_frag)

      Query: mystery_seq <unknown description>
        Hit: chr19 <unknown description>
Query range: [0:61] (1)
  Hit range: [54204480:54204541] (1)
  Fragments: 1 (? columns)



In all cases, these attributes are accessible using our favorite dot
notation. Some examples:



In [69]:
blast_frag.query_start      # query start coordinate

86

In [70]:
blast_frag.hit_strand       # hit sequence strand

1

In [71]:
blast_frag.hit              # hit sequence, as a SeqRecord object

SeqRecord(seq=Seq('TTGGCCATGAAAACTGAGCAAATGGCGTTGGCTAATTTGATAGATTATGATATG...TAG', DNAAlphabet()), id='gi|731339628|ref|XM_010682658.1|', name='aligned hit sequence', description='PREDICTED: Beta vulgaris subsp. vulgaris cold-regulated 413 plasma membrane protein 2 (LOC104895996), mRNA', dbxrefs=[])


A note about standards and conventions {#sec:searchio-standards}
--------------------------------------

Before we move on to the main functions, there is something you ought to
know about the standards `Bio.SearchIO` uses. If you’ve worked with
multiple sequence search tools, you might have had to deal with the many
different ways each program deals with things like sequence coordinates.
It might not have been a pleasant experience as these search tools
usually have their own standards. For example, one tools might use
one-based coordinates, while the other uses zero-based coordinates. Or,
one program might reverse the start and end coordinates if the strand is
minus, while others don’t. In short, these often creates unnecessary
mess must be dealt with.

We realize this problem ourselves and we intend to address it in
`Bio.SearchIO`. After all, one of the goals of `Bio.SearchIO` is to
create a common, easy to use interface to deal with various search
output files. This means creating standards that extend beyond the
object model you just saw.

Now, you might complain, “Not another standard!”. Well, eventually we
have to choose one convention or the other, so this is necessary. Plus,
we’re not creating something entirely new here; just adopting a standard
we think is best for a Python programmer (it is Biopython, after all).

There are three implicit standards that you can expect when working with
`Bio.SearchIO`:

-   The first one pertains to sequence coordinates. In `Bio.SearchIO`,
    all sequence coordinates follows Python’s coordinate style:
    zero-based and half open. For example, if in a BLAST XML output file
    the start and end coordinates of an HSP are 10 and 28, they would
    become 9 and 28 in `Bio.SearchIO`. The start coordinate becomes 9
    because Python indices start from zero, while the end coordinate
    remains 28 as Python slices omit the last item in an interval.

-   The second is on sequence coordinate orders. In `Bio.SearchIO`,
    start coordinates are always less than or equal to end coordinates.
    This isn’t always the case with all sequence search tools, as some
    of them have larger start coordinates when the sequence strand
    is minus.

-   The last one is on strand and reading frame values. For strands,
    there are only four valid choices: `1` (plus strand), `-1` (minus
    strand), `0` (protein sequences), and `None` (no strand). For
    reading frames, the valid choices are integers from `-3` to `3` and
    `None`.

Note that these standards only exist in `Bio.SearchIO` objects. If you
write `Bio.SearchIO` objects into an output format, `Bio.SearchIO` will
use the format’s standard for the output. It does not force its standard
over to your output file.

Reading search output files {#sec:searchio-input}
---------------------------

There are two functions you can use for reading search output files into
`Bio.SearchIO` objects: `read` and `parse`. They’re essentially similar
to `read` and `parse` functions in other submodules like `Bio.SeqIO` or
`Bio.AlignIO`. In both cases, you need to supply the search output file
name and the file format name, both as Python strings. You can check the
documentation for a list of format names `Bio.SearchIO` recognizes.

`Bio.SearchIO.read` is used for reading search output files with only
one query and returns a `QueryResult` object. You’ve seen `read` used in
our previous examples. What you haven’t seen is that `read` may also
accept additional keyword arguments, depending on the file format.

Here are some examples. In the first one, we use `read` just like
previously to read a BLAST tabular output file. In the second one, we
use a keyword argument to modify so it parses the BLAST tabular variant
with comments in it:



In [73]:
from Bio import SearchIO
qresult = SearchIO.read('data/tab_2226_tblastn_003.txt', 'blast-tab')
qresult

QueryResult(id='gi|16080617|ref|NP_391444.1|', 3 hits)

In [75]:
qresult2 = SearchIO.read('data/tab_2226_tblastn_007.txt', 'blast-tab', comments=True)
qresult2

QueryResult(id='gi|16080617|ref|NP_391444.1|', 3 hits)


These keyword arguments differs among file formats. Check the format
documentation to see if it has keyword arguments that modifies its
parser’s behavior.

As for the `Bio.SearchIO.parse`, it is used for reading search output
files with any number of queries. The function returns a generator
object that yields a `QueryResult` object in each iteration. Like
`Bio.SearchIO.read`, it also accepts format-specific keyword arguments:



In [77]:
from Bio import SearchIO
qresults = SearchIO.parse('data/tab_2226_tblastn_001.txt', 'blast-tab')
for qresult in qresults:
    print(qresult.id)

gi|16080617|ref|NP_391444.1|
gi|11464971:4-101


In [78]:
qresults2 = SearchIO.parse('data/tab_2226_tblastn_005.txt', 'blast-tab', comments=True)
for qresult in qresults2:
    print(qresult.id)

random_s00
gi|16080617|ref|NP_391444.1|
gi|11464971:4-101



Dealing with large search output files with indexing {#sec:searchio-index}
----------------------------------------------------

Sometimes, you’re handed a search output file containing hundreds or
thousands of queries that you need to parse. You can of course use
`Bio.SearchIO.parse` for this file, but that would be grossly
inefficient if you need to access only a few of the queries. This is
because `parse` will parse all queries it sees before it fetches your
query of interest.

In this case, the ideal choice would be to index the file using
`Bio.SearchIO.index` or `Bio.SearchIO.index_db`. If the names sound
familiar, it’s because you’ve seen them before in
Section \[sec:SeqIO-index\]. These functions also behave similarly to
their `Bio.SeqIO` counterparts, with the addition of format-specific
keyword arguments.

Here are some examples. You can use `index` with just the filename and
format name:



In [79]:
from Bio import SearchIO
idx = SearchIO.index('data/tab_2226_tblastn_001.txt', 'blast-tab')
sorted(idx.keys())

['gi|11464971:4-101', 'gi|16080617|ref|NP_391444.1|']

In [80]:
idx['gi|16080617|ref|NP_391444.1|']

QueryResult(id='gi|16080617|ref|NP_391444.1|', 3 hits)

In [81]:
idx.close()


Or also with the format-specific keyword argument:



In [82]:
idx = SearchIO.index('data/tab_2226_tblastn_005.txt', 'blast-tab', comments=True)
sorted(idx.keys())

['gi|11464971:4-101', 'gi|16080617|ref|NP_391444.1|', 'random_s00']

In [83]:
idx['gi|16080617|ref|NP_391444.1|']

QueryResult(id='gi|16080617|ref|NP_391444.1|', 3 hits)

In [84]:
idx.close()


Or with the `key_function` argument, as in `Bio.SeqIO`:



In [85]:
key_function = lambda id: id.upper()    # capitalizes the keys
idx = SearchIO.index('data/tab_2226_tblastn_001.txt', 'blast-tab', key_function=key_function)
sorted(idx.keys())

['GI|11464971:4-101', 'GI|16080617|REF|NP_391444.1|']

In [86]:
idx['GI|16080617|REF|NP_391444.1|']

QueryResult(id='gi|16080617|ref|NP_391444.1|', 3 hits)

In [87]:
idx.close()


`Bio.SearchIO.index_db` works like as `index`, only it writes the query
offsets into an SQLite database file.

Writing and converting search output files {#sec:searchio-write}
------------------------------------------

It is occasionally useful to be able to manipulate search results from
an output file and write it again to a new file. `Bio.SearchIO` provides
a `write` function that lets you do exactly this. It takes as its
arguments an iterable returning `QueryResult` objects, the output
filename to write to, the format name to write to, and optionally some
format-specific keyword arguments. It returns a four-item tuple, which
denotes the number or `QueryResult`, `Hit`, `HSP`, and `HSPFragment`
objects that were written.



In [88]:
from Bio import SearchIO
qresults = SearchIO.parse('data/mirna.xml', 'blast-xml')     # read XML file
SearchIO.write(qresults, 'results.tab', 'blast-tab')    # write to tabular file

(3, 239, 277, 277)


You should note different file formats require different attributes of
the `QueryResult`, `Hit`, `HSP` and `HSPFragment` objects. If these
attributes are not present, writing won’t work. In other words, you
can’t always write to the output format that you want. For example, if
you read a BLAST XML file, you wouldn’t be able to write the results to
a PSL file as PSL files require attributes not calculated by BLAST (e.g.
the number of repeat matches). You can always set these attributes
manually, if you really want to write to PSL, though.

Like `read`, `parse`, `index`, and `index_db`, `write` also accepts
format-specific keyword arguments. Check out the documentation for a
complete list of formats `Bio.SearchIO` can write to and their
arguments.

Finally, `Bio.SearchIO` also provides a `convert` function, which is
simply a shortcut for `Bio.SearchIO.parse` and `Bio.SearchIO.write`.
Using the convert function, our example above would be:



In [89]:
from Bio import SearchIO
SearchIO.convert('data/mirna.xml', 'blast-xml', 'results.tab', 'blast-tab')

(3, 239, 277, 277)